In [ ]:
%cd /content/drive/MyDrive/Malaria

/content/drive/MyDrive/Malaria


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
df=pd.read_json('malaria/training.json')

In [ ]:
df.head()

,image,objects
0,{'checksum': 'eea3bfd6a929bcb06f9786667cd3fbb2...,"[{'bounding_box': {'minimum': {'r': 576, 'c': ..."
1,{'checksum': '2d85fc8ffdb875bb569878243437e4f5...,"[{'bounding_box': {'minimum': {'r': 1048, 'c':..."
2,{'checksum': 'f7c02b6247f3ccfcd4a22c1f06e0c72c...,"[{'bounding_box': {'minimum': {'r': 979, 'c': ..."
3,{'checksum': '50634ef166ae993b4562b96f315ff6db...,"[{'bounding_box': {'minimum': {'r': 402, 'c': ..."
4,{'checksum': '74dd777a74e5a4b24aaee574a99e1459...,"[{'bounding_box': {'minimum': {'r': 314, 'c': ..."


Extracting Images Informations

In [ ]:
df["ImagePath"] = df['image'].apply(lambda x: x["pathname"][1:])

In [ ]:
df["shape"] = df['image'].apply(lambda x: x["shape"])

In [ ]:
df.sample()

,image,objects,ImagePath,shape
57,{'checksum': '10ad46ddf60fff31021e74a5f1db8840...,"[{'bounding_box': {'minimum': {'r': 335, 'c': ...",images/a1265844-329c-4a49-a37f-e0541b83a50c.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}"


In [ ]:
T=[]
for i,row in df.iterrows():
  for bb in row['objects']:
    temp={
        "ImagePAth":row['ImagePath'],
        "Shape": row['shape'],
        **bb
    }
    T.append(temp)


In [ ]:
data=pd.DataFrame(T)
data.sample(5)

,ImagePAth,Shape,bounding_box,category
5734,images/e26b88c6-5884-4b5c-94d7-ad780379c8ad.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}","{'minimum': {'r': 159, 'c': 666}, 'maximum': {...",red blood cell
924,images/8874ea02-d263-4830-99d1-e1b14230a56b.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}","{'minimum': {'r': 1004, 'c': 595}, 'maximum': ...",red blood cell
2997,images/c736848a-437c-4121-b99f-58f186eb3a1d.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}","{'minimum': {'r': 1110, 'c': 688}, 'maximum': ...",trophozoite
4622,images/74bf1267-d98b-4532-a81d-9d8df63e0f75.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}","{'minimum': {'r': 1196, 'c': 1695}, 'maximum':...",red blood cell
5297,images/dea9006d-c360-4118-b77b-8bc959fa5f8a.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}","{'minimum': {'r': 781, 'c': 719}, 'maximum': {...",red blood cell


In [ ]:
def extract_nested_list(it):
    if isinstance(it, list):
        for sub_it in it:
            yield from extract_nested_list(sub_it)
    elif isinstance(it, dict):
        for value in it.values():
            yield from extract_nested_list(value)
    else:
        yield it

In [ ]:
data['x_min'] = -1
data['y_min'] = -1
data['x_max'] = -1
data['y_max'] = -1

data[['y_min', 'x_min', 'y_max', 'x_max']] = np.stack(data['bounding_box'].apply(lambda x: np.array(list(extract_nested_list(x)))))
data.drop(columns=['bounding_box'], inplace=True)
data['x_min'] = data['x_min'].astype(float)
data['y_min'] = data['y_min'].astype(float)
data['x_max'] = data['x_max'].astype(float)
data['y_max'] = data['y_max'].astype(float)

data['w'] = data['x_max'] - data['x_min']
data['h'] = data['y_max'] - data['y_min']

data['x_center'] = data['x_min'] + data['w'] / 2
data['y_center'] = data['y_min'] + data['h'] / 2

In [ ]:
data.sample(10)

,ImagePAth,Shape,category,x_min,y_min,x_max,y_max,w,h,x_center,y_center
2247,images/351d6536-3a5c-46eb-a6e4-d71dd999908a.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",red blood cell,402.0,1156.0,528.0,1280.0,126.0,124.0,465.0,1218.0
1264,images/baab0701-3fd4-4940-8b90-b9c42d95888f.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",red blood cell,1182.0,912.0,1322.0,1041.0,140.0,129.0,1252.0,976.5
352,images/ebc4056e-d766-43cf-b6b4-5dac79d96e84.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",red blood cell,1054.0,626.0,1215.0,746.0,161.0,120.0,1134.5,686.0
4150,images/9fd32265-07f2-442b-80b5-35069891d452.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",ring,1069.0,33.0,1220.0,161.0,151.0,128.0,1144.5,97.0
1739,images/1e647046-333b-4102-a0c6-4c29405c37b3.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",red blood cell,859.0,643.0,1002.0,781.0,143.0,138.0,930.5,712.0
1129,images/511b6e13-acd7-4dd3-b1d0-c4aa10a514c7.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",red blood cell,554.0,90.0,707.0,234.0,153.0,144.0,630.5,162.0
1946,images/bdfbad30-fa2f-4285-bf02-6064ab6d869c.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",red blood cell,1376.0,869.0,1514.0,1005.0,138.0,136.0,1445.0,937.0
113,images/64985a1e-16bb-4016-a01c-c21a5b86e572.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",red blood cell,162.0,1059.0,327.0,1212.0,165.0,153.0,244.5,1135.5
2117,images/8e1e0fd3-52fb-4ebb-82de-67bbd66fcf4d.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",red blood cell,60.0,453.0,197.0,543.0,137.0,90.0,128.5,498.0
359,images/ebc4056e-d766-43cf-b6b4-5dac79d96e84.jpg,"{'r': 1383, 'c': 1944, 'channels': 3}",red blood cell,848.0,897.0,957.0,1020.0,109.0,123.0,902.5,958.5


In [ ]:
data['img_width'] = -1
data['img_height'] = -1
data['channels'] = -1
data['pixels'] = -1

data[['img_height','img_width', 'channels']] = np.stack(data['Shape'].apply(lambda x: np.array(list(extract_nested_list(x)))))
data.drop(columns=['Shape'], inplace=True)
data['pixels'] = data['img_width'] * data['img_height'] * data['channels']

In [ ]:
data.sample(5)

,ImagePAth,category,x_min,y_min,x_max,y_max,w,h,x_center,y_center,img_width,img_height,channels,pixels
5113,images/9cf3f75a-354c-46f9-8b53-0a9711ac8bee.jpg,red blood cell,847.0,84.0,979.0,218.0,132.0,134.0,913.0,151.0,1944,1383,3,8065656
1841,images/a3548f89-1cb5-4e67-b460-482df95b10ee.jpg,red blood cell,1335.0,493.0,1463.0,623.0,128.0,130.0,1399.0,558.0,1944,1383,3,8065656
4612,images/74bf1267-d98b-4532-a81d-9d8df63e0f75.jpg,red blood cell,1766.0,1103.0,1899.0,1229.0,133.0,126.0,1832.5,1166.0,1944,1383,3,8065656
4600,images/a4bd99a0-226a-4044-914b-8073d5890734.jpg,red blood cell,1553.0,1256.0,1691.0,1376.0,138.0,120.0,1622.0,1316.0,1944,1383,3,8065656
2154,images/0897f682-5301-49fc-a8d5-9df7762040bb.jpg,red blood cell,1652.0,395.0,1782.0,514.0,130.0,119.0,1717.0,454.5,1944,1383,3,8065656


In [ ]:
data.to_csv('train_data.csv', index=False)